In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from urllib.parse import quote
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
# Connect SQL Server
def get_data_from_sql_server():
    server = 'VM-DC-JUMPSRV77\IFRS9'
    database = 'EWS'
    user = 'rdm_admin'
    pw = '2024#tpb'
    driver = '{SQL Server}'
    cnxn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={user};PWD={pw}"
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quote(cnxn_str)}")
    # query
    query = "SELECT * FROM [EWS].[dbo].[ews_khdn_score_store] where model = 'B-score'"
    df = pd.read_sql(query, engine)
    return df

In [3]:
def calculate_ks(expected, actual):
    ks_statistic, p_value = ks_2samp(expected, actual)
    return ks_statistic, p_value   

In [6]:
import numpy as np
import warnings

def calculate_psi(expected, actual, categorical=False, bins=None):
    # Check if the variables are categorical
    if categorical:
        # Get unique categories
        categories = np.unique(np.concatenate([expected, actual]))

        # Issue a warning if the number of unique categories exceeds 20
        if len(categories) > 20:
            warnings.warn("Warning: Number of unique categories exceeds 20.")

        # Calculate the expected and actual proportions for each category
        expected_probs = np.array([np.sum(expected == cat) for cat in categories]) / len(expected)
        actual_probs = np.array([np.sum(actual == cat) for cat in categories]) / len(actual)
    else:
        expected = expected[~np.isnan(expected)]
        actual = actual[~np.isnan(actual)]
        # Apply binning for numeric columns
        if bins is None:
            bins = 10  # Default to 10 bins, you can change this value as needed

        # Calculate the bin edges based on percentiles
        #bin_edges = np.percentile(np.hstack((expected, actual)), np.linspace(0, 100, bins + 1))
        bin_edges = np.linspace(min(min(expected), min(actual)), max(max(expected), max(actual)), 11)
        # Calculate the expected and actual proportions for each bin
        expected_probs, _ = np.histogram(expected, bins=bin_edges)
        actual_probs, _ = np.histogram(actual, bins=bin_edges)

        # Normalize to get proportions
        expected_probs = expected_probs / len(expected)
        actual_probs = actual_probs / len(actual)
        
    # Initialize PSI
    psi_value = 0

    # Loop over each bin or category
    for i in range(len(expected_probs)):
        # Avoid division by zero and log of zero
        if expected_probs[i] == 0 or actual_probs[i] == 0:
            continue
        # Calculate the PSI for this bin or category
        psi_value += (expected_probs[i] - actual_probs[i]) * np.log(expected_probs[i] / actual_probs[i])

    return psi_value

In [7]:
def calculate_psi_ks_by_process_date(df, quantitative_columns, categorical_columns):
    psi_ks_table = pd.DataFrame(columns=['process_date', 'criteria', 'variable', 'value', 'ks_pvalue'])
    df['process_date'] = pd.to_datetime(df['process_date'])
    end_of_month_dates = pd.date_range(start='2023-10-31', end='2023-12-31', freq='M')
    for process_date in end_of_month_dates:
        # 
        data_before = df[(df['process_date'].dt.month == 12) & (df['process_date'].dt.year >= 2013) & (df['process_date'].dt.year <= 2019)]
        data_after = df[df['process_date'] == process_date]
        #
        for column in categorical_columns:
            actual = data_before[column].values
            expected = data_after[column].values
            psi_score = calculate_psi(expected, actual, categorical=True, bins=10)
            psi_ks_table = psi_ks_table.append({'process_date': process_date,
                                                'criteria': 'PSI',
                                                'variable': column,
                                                'value': psi_score},
                                                ignore_index=True) 

        for column in quantitative_columns:
            actual = data_before[column].values
            expected = data_after[column].values
            psi_score = calculate_psi(expected, actual, categorical=False, bins=10)
            psi_ks_table = psi_ks_table.append({'process_date': process_date,
                                                'criteria': 'PSI',
                                                'variable': column,
                                                'value': psi_score},
                                                ignore_index=True)                                            

        for column in quantitative_columns:
            actual = data_after[column].values
            expected = data_before[column].values
            ks_statistic, p_value = calculate_ks(expected, actual)
            psi_ks_table = psi_ks_table.append({'process_date': process_date,
                                                'criteria': 'KS',
                                                'variable': column,
                                                'value': ks_statistic,
                                                'ks_pvalue': p_value},
                                                ignore_index=True)
    
    return psi_ks_table
df = get_data_from_sql_server()
# List các cột
categorical_columns = ['grade']
quantitative_columns = ['score',
   'min_ca_bal_l6m'
  ,  'vol_os_l6m'
  ,  'num_cr_product_l3m'
  ,  'dpd'
  ,  'num_xdpd_l6m'
  ,  'total_bal_os_onbs_avg'
  ,  'avg_ebank_txn_amt_l3m'
  ,  'avg_cash_txn_amt_per_time_l3m'
  ,  'total_bal_os_onbs_avg_woe'
  ,  'min_ca_bal_l6m_woe'
  ,  'vol_os_l6m_woe'
  ,  'num_xdpd_l6m_woe'
  ,  'dpd_woe'
  ,  'avg_ebank_txn_amt_l3m_woe'
  ,  'num_cr_product_l3m_woe'
  ,  'avg_cash_txn_amt_per_time_l3m_woe'
     ]

result_table = calculate_psi_ks_by_process_date(df, quantitative_columns, categorical_columns)
print(result_table)

    process_date criteria                           variable     value  \
0     2023-10-31      PSI                              grade  0.214822   
1     2023-10-31      PSI                              score  0.015627   
2     2023-10-31      PSI                     min_ca_bal_l6m  0.000017   
3     2023-10-31      PSI                         vol_os_l6m  0.034919   
4     2023-10-31      PSI                 num_cr_product_l3m  0.046908   
..           ...      ...                                ...       ...   
100   2023-12-31       KS                   num_xdpd_l6m_woe  0.069071   
101   2023-12-31       KS                            dpd_woe  0.031376   
102   2023-12-31       KS          avg_ebank_txn_amt_l3m_woe  0.340203   
103   2023-12-31       KS             num_cr_product_l3m_woe  0.090125   
104   2023-12-31       KS  avg_cash_txn_amt_per_time_l3m_woe  0.253500   

        ks_pvalue  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4          

In [8]:
result_table.to_excel('Z:/Noibo/PHONG MHRR/LanVH/PSI Bscore.xlsx', index= False)